In [1]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Define a function to extract MFCC features from audio files
def extract_mfcc(audio_path, max_pad_len=174):
    audio, _ = librosa.load(audio_path, sr=None)
    mfccs = librosa.feature.mfcc(audio, sr=16000, n_mfcc=13)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfccs

# Define the path to your dataset
data_dir = 'path_to_your_dataset'
emotion_labels = ['angry', 'happy', 'neutral', 'sad']

# Load and preprocess the dataset
X = []
y = []
for emotion_label in emotion_labels:
    emotion_folder = os.path.join(data_dir, emotion_label)
    for audio_file in os.listdir(emotion_folder):
        audio_path = os.path.join(emotion_folder, audio_file)
        mfccs = extract_mfcc(audio_path)
        X.append(mfccs)
        y.append(emotion_label)

X = np.array(X)
y = np.array(y)

# Encode the emotion labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Build a simple CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(13, 174, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(emotion_labels), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

# Save the model
model.save('speech_emotion_recognition_model.h5')


ModuleNotFoundError: No module named 'librosa'